In [ ]:
# the following lines disable the numpy multithreading [optional]
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import KitBase as kt

import matplotlib.pyplot as plt

from examples.nonlinear.boltzmann_3d_pbc_upwind1 import Boltzmann
prob = Boltzmann()

In [ ]:
# for Jupyter
prob.document = 'None'
prob.betas = [0]

In [ ]:
# choosing a number of points
prob.spatial_points = [30, 15, 10, 10]
prob.time_points = 1

# choosing a time domain
prob.T_start = 0

# choosing the number of intervals handled in parallel
prob.time_intervals = 1
prob.rolling = 20

prob.T_end = 1e-3 * prob.time_intervals * prob.rolling

# choosing a parallelization strategy
prob.proc_col = 1
prob.proc_row = prob.time_intervals

# choosing a solver
prob.solver = 'gmres'

# setting maximum number of iterations
prob.maxiter = 50
prob.smaxiter = 50

# choosing a setting for the alpha sequence
prob.alphas = [1e-6]

# setting tolerances
prob.tol = 1e-6
prob.stol = 1e-8

prob.setup()
prob.solve()
prob.summary(details=True)

In [ ]:
import numpy as np
prob.__fill_u_last__(fill_old=False)

w0 = np.zeros((prob.spatial_points[0], 5))
w = np.zeros_like(w0)

f0 = prob.u_initial(0)
f = prob.u_last_loc.reshape(prob.spatial_points)

for i in range(prob.spatial_points[0]):
    w0[i, :] = kt.moments_conserve(f0[i, :, :, :].real, prob.vs.u, prob.vs.v, prob.vs.w, prob.vs.weights)
    w[i, :] = kt.moments_conserve(f[i, :, :, :].real, prob.vs.u, prob.vs.v, prob.vs.w, prob.vs.weights)
    
plt.plot(w[:, 1])
plt.plot(w0[:, 1], '--')
plt.legend(['w', 'w0'])
plt.savefig('w_and_w0.png', dpi=500)
plt.show()
    